# Implementation of  LSH from scratch

In this notebook,we will implement LSH from scratch and predict the labels of the test data. we then verify the correctness of the implementation using a "grader" function/cell which will match the implmentation.

The grader fucntion would help to validate the correctness of the code. 


## Reading the data from csv file

In [13]:
import numpy as np

In [31]:
# Loading data from csv file
import pandas as pd
data_path = 'lsh_data.csv'

df = pd.read_csv(data_path)
df['text']

0       tv future in the hands of viewers with home th...
1       worldcom boss  left books alone  former worldc...
2       tigers wary of farrell  gamble  leicester say ...
3       yeading face newcastle in fa cup premiership s...
4       ocean s twelve raids box office ocean s twelve...
                              ...                        
2220    cars pull down us retail figures us retail sal...
2221    kilroy unveils immigration policy ex-chatshow ...
2222    rem announce new glasgow concert us band rem h...
2223    how political squabbles snowball it s become c...
2224    souness delight at euro progress boss graeme s...
Name: text, Length: 2225, dtype: object

In [15]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [16]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [17]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


## Custom Implementation

### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **4000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate **5 random hyperplanes**. And generate the hyperplanes using a random normal distribution with **mean zero and variance 1**. 
  6. i have set the **numpy random seed to zero**, we will use the  **np.random.normal** to generate the vectors for hyperplanes.

  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute **11-Nearest Neighbours** from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. **In case of a tie** in the obtained labels from nearest neighbours, we can pick a label after sorting all the labels **alphabetically**(A-Z), i.e. for example labels starting with A would get more preference than labels starting with Z.
  14. Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list so  return should be of length 10.

In [18]:
#importing the tfidf vectorizer from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
# initialising tfidf vectorizer with parameters mentioned above
vectorizer = TfidfVectorizer( ngram_range=(2, 3),max_features=4000,min_df=10)  
X_train = vectorizer.fit_transform(train_data['text'])

In [20]:
#defining all the five planes p1,p2,p3,p4,p4
trainmatrix=X_train.toarray() #converting sparsematrix to array
np.random.seed(0)
p1=np.random.normal(0,1,4000)
p2=np.random.normal(0,1,4000)
p3=np.random.normal(0,1,4000)
p4=np.random.normal(0,1,4000)
p5=np.random.normal(0,1,4000)

In [21]:
#creating hashtable for storing key as hashvalue generated using planes and value as list of indexes of the train data 
#indexes are stored  as value to get the labels in the next steps
table={}
labeltalbe={}
for j,i in enumerate(trainmatrix):
    sign1,sign2,sign3,sign4,sign5=0,0,0,0,0
    if np.dot(i.T,p1) >=0:sign1=1
    if np.dot(i.T,p2) >=0:sign2=1
    if np.dot(i.T,p3) >=0:sign3=1
    if np.dot(i.T,p4) >=0:sign4=1
    if np.dot(i.T,p5) >=0:sign5=1
    hashval=(sign1,sign2,sign3,sign4,sign5)
    if hashval in table.keys():
        table[hashval].append(j)
    else:
        table[hashval]=[j]

In [22]:
def vect_val(x):
    '''
    Function for calculating magnitude of the vector
    '''
    sqr=sum([i**2 for i in x])
    return sqr**0.5
def cossm(x,y):
    '''
    Calculating Cosine-similarity
    
    '''
    try:
        return np.dot(x.T,y)/(vect_val(x)*vect_val(y))
    except:
        return 0

In [25]:
# Please implement this fucntion and write your code wherever asked. Do NOT change the code snippets provided by us.

def predictLabels (test_data):
    """
    Given the test_data, return the labels for all the rows in the test data.
    Follow the step by step instructions mentioned above.

    """
    X_test=vectorizer.transform(test_data['text'])
    testtable={}
    testmatrix=X_test.toarray()
    output=[]
    for j,i in enumerate(testmatrix):
        sign1,sign2,sign3,sign4,sign5=0,0,0,0,0
        if np.dot(i.T,p1) >= 0:sign1=1
        if np.dot(i.T,p2) >= 0:sign2=1
        if np.dot(i.T,p3) >= 0:sign3=1
        if np.dot(i.T,p4) >= 0:sign4=1
        if np.dot(i.T,p5) >= 0:sign5=1
        hashval=(sign1,sign2,sign3,sign4,sign5)
        testtable[j]=hashval
        
    for j,i in enumerate(testmatrix):
        neighbors=table[testtable[j]]
        dis=[]
        index=[]
        for nb in neighbors:
            dis.append(cossm(i,trainmatrix[nb]))
            index.append(nb)
        res=pd.DataFrame({'dis':dis,'index':index}).sort_values(by='dis',ascending=False).head(11).reset_index(drop=True)
        classes=train_data['category'][res['index']].value_counts()
        output.append(classes.index[0])
    return output 

## Grader Cell

Executing the following Grader cell to verify the correctness the above implementation. This cell will print "Success" if implmentation of the predictLabels() is correct, else, it will print "Failed".

In [26]:
###########################################
## GRADER CELL:
# This cell will print "Success" if the implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
  print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
  print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
  print("\nY_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\nY_custom = \n\n", Y_custom)


******** Success ******** Accuracy Achieved =  90 %
